In [ ]:
%env AWS_REQUEST_PAYER=requester

In [ ]:
from stacLib.ard_stac_search import Astac
import rasterio as rio
from rasterio.windows import Window

import numpy as np
import tqdm.notebook as tq
#import zarr

In [ ]:
#my_date='2020-01-01/2021-12-31'
my_date='2020-01-01/2020-02-28'

In [ ]:
stac_config = {}

stac_config['url'] = 'https://landsatlook.usgs.gov/stac-server'
stac_config['fav_collection'] = 'landsat-c2l3-dswe'
stac_config['description'] = 'Landsat Collection 2 ARD Tiles DSWE'


In [ ]:
H='21'
V='10'
YEAR=2021

In [ ]:
_stac = Astac(stac_config)

In [ ]:
date_range_text = my_date
cloud_cover_pct_max = '100'
_assets = _stac.get_my_assets(None, H, V, date_range_text, cloud_cover_pct_max)

In [ ]:
_assets[0]['properties']

In [ ]:

def read_tif(path: str, band: int = 1, window: Window = None):
    return rio.open(path).read(band, window=window)

def read_ls_data(item, band, window):
    return read_tif(item['assets'][band]['alternate']['s3']['href'], window=window)

In [ ]:
len(_assets)

In [ ]:
# Initialize 3d array time Y X

In [ ]:
window = Window(0, 0, 256, 256)

In [ ]:
dswe_array = np.zeros(shape=(len(_assets), 256, 256), dtype=np.uint16)

In [ ]:
dswe_array.shape

In [ ]:
%%time
p_bar=tq.tqdm(range(len(_assets)), position=0, leave=True)
myband='intsm'
for t1 in range(len(_assets)):
    item = _assets[t1]
    dswe_array[t1] = read_ls_data(item, myband, window)
    p_bar.update(1)
    p_bar.refresh()

In [ ]:
my_dates=[]
for t1 in range(len(_assets)):
    print(t1)
    my_date = _assets[t1]['properties']['datetime']
    my_dates.append(my_date)

In [ ]:
_assets[0]['properties']['datetime']

In [ ]:
my_dates

In [ ]:
import holoviews as hv
hv.extension('bokeh', 'matplotlib')

In [ ]:
dswe_array.shape

In [ ]:
mask255 = (dswe_array == 255)
dswe_array[mask255] = 5

ds = hv.Dataset((np.arange(256), np.arange(256), np.arange(10), big3d_ary), ['Time', 'y', 'x'], 'INTSM')

In [ ]:
from holoviews import opts

In [ ]:
opts.defaults(
    opts.GridSpace(shared_xaxis=True, shared_yaxis=True),
    opts.Image(cmap='tab20', width=400, height=400),
    opts.Labels(text_color='white', text_font_size='8pt', text_align='left', text_baseline='bottom'),
    opts.Path(color='white'),
    opts.Spread(width=600),
    opts.Overlay(show_legend=False))

# 99 FAIL - the next step takes forever 

In [ ]:
import numpy as np
import xarray as xr

# Assuming dswe_array is your 3D NumPy array
# Replace the dimensions, time, y, x, and data variable (dswe) names accordingly

# Example dimensions
time_dim = np.arange(dswe_array.shape[0])
y_dim = np.arange(dswe_array.shape[1])
x_dim = np.arange(dswe_array.shape[2])

# Create xarray DataArray
dswe_xr = xr.DataArray(dswe_array, dims=('time', 'y', 'x'),
                       coords={'time': time_dim, 'y': y_dim, 'x': x_dim},
                       name='dswe')

# You can also add attributes if needed
dswe_xr.attrs['units'] = 'some_unit'

# Print the xarray DataArray
print(dswe_xr)


In [ ]:
import hvplot.xarray

In [ ]:
dswe_xr.hvplot.image(rasterize=True,  x='x', y='y', width=500, height=400, cmap='tab20' ).hist()

In [ ]:
#ds = dswe_xr.to_dataset()

In [ ]:
#ds.hvplot.image(rasterize=True,  x='x', y='y', width=500, height=400, cmap='tab20' )